In [1]:
import fxcmpy
import numpy as np
import time

In [2]:
# function to get info about current open positions - currency, amount and side
def get_positions():
    for i in range(len(con.get_open_positions(kind='list'))):
        print(f"{con.get_open_positions(kind='list')[i]['currency']}: {con.get_open_positions(kind='list')[i]['amountK']} - Long: {con.get_open_positions(kind='list')[i]['isBuy']}")

In [3]:
# test instruments
instruments = ['EUR/USD', 'USD/JPY', 'AUD/USD', 'GBP/USD']
target_positions =  [10.00,  0.00,  35.00, -40.00]
current_positions = [10.00, 20.00, -30.00,  45.00]
TIME_TO_SLEEP = 5

In [4]:
# connecting and closing all positions
TOKEN = 'ab0d332e362719ca0474b276adead6b7efafd1da'
SERVER = 'demo'
con = fxcmpy.fxcmpy(access_token=TOKEN, log_file='log.txt', log_level='debug', server=SERVER)

con.close_all()
time.sleep(TIME_TO_SLEEP)
get_positions()

In [5]:
# opening current positions for the test
for i in range(len(instruments)):
    if current_positions[i] > 0:
        con.create_market_buy_order(symbol=instruments[i], amount=current_positions[i])
    elif current_positions[i] < 0:
        con.create_market_sell_order(symbol=instruments[i], amount=np.abs(current_positions[i]))
    time.sleep(TIME_TO_SLEEP)

In [6]:
# check current positions
get_positions()

EUR/USD: 10 - Long: True
USD/JPY: 20 - Long: True
AUD/USD: 30 - Long: False
GBP/USD: 45 - Long: True


In [7]:
for i, instrument in enumerate(instruments):

    # EUR/USD expected to remain the same as they are 10 in both target positions and current positions
    if target_positions[i] == current_positions[i]:
        print('EUR/USD expected to remain the same as they are 10 in both target positions and current positions')
        get_positions()
        continue

    # USD/JPY expected to get closed as it is 0 in target positions
    elif target_positions[i] == 0:
        print('\nUSD/JPY expected to get closed as it is 0 in target positions')
        con.close_all_for_symbol(instrument)
        time.sleep(TIME_TO_SLEEP)
        get_positions()
        continue

    # AUD/USD short position expected to be closed and long position to be opened
    elif current_positions[i] < 0 and target_positions[i] > 0:
        print('\nAUD/USD short position expected to be closed and long position to be opened')
        con.close_all_for_symbol(instrument)
        time.sleep(TIME_TO_SLEEP)
        order = None
        while order is None:
            order = con.create_market_buy_order(instrument, np.abs(target_positions[i]))
        print('Buy ' + instrument + ' ' + str(np.abs(target_positions[i])))
        get_positions()
        continue

    # GBP/USD long position expected to be closed and short position to be open
    elif current_positions[i] > 0 and target_positions[i] < 0:
        print('\nGBP/USD long position expected to be closed and short position to be open')
        con.close_all_for_symbol(instrument)
        time.sleep(TIME_TO_SLEEP)
        order = None
        while order is None:
            order = con.create_market_sell_order(instrument, np.abs(target_positions[i]))
        print('Sell ' + instrument + ' ' + str(np.abs(target_positions[i])))
        get_positions()
        continue

EUR/USD expected to remain the same as they are 1000 in both target positions and current positions
EUR/USD: 10 - Long: True
USD/JPY: 20 - Long: True
AUD/USD: 30 - Long: False
GBP/USD: 45 - Long: True

USD/JPY expected to get closed as it is 0 in target positions
EUR/USD: 10 - Long: True
AUD/USD: 30 - Long: False
GBP/USD: 45 - Long: True

AUD/USD short position expected to be closed and long position to be opened
Buy AUD/USD 35.0
EUR/USD: 10 - Long: True
GBP/USD: 45 - Long: True
AUD/USD: 35 - Long: True

GBP/USD long position expected to be closed and short position to be open
Sell GBP/USD 40.0
EUR/USD: 10 - Long: True
AUD/USD: 35 - Long: True
GBP/USD: 40 - Long: False
